1.	How can I set the PATH variable for a task or any other environment variable?

The environment variables can be set by using the ‘environment’ keyword. It can be set for either a task or an entire playbook as well. Follow the below code snippet to see how:

environment:
PATH: "{{ ansible_env.PATH }}:/thingy/bin"
SOME: value

2.	How can encrypted passwords for the user module be generated?

The simplest way is to use the ad-hoc command:

ansible all -i localhost, -m debug -a "msg={{ 'mypassword' | password_hash('sha512', 'mysecretsalt') }}"

3.	How can a list of hosts in a group be looped over within a template?

An easy way to do this is to iterate over a list of hosts inside of a host group, in order to fill a template configuration file with a list of servers. This can be done by accessing the “$groups” dictionary in your template, like so:

{% for host in groups['db_servers'] %}
{{ host }}
{% endfor %}
In order to access facts about these hosts, like, the IP address of each hostname, you need to make sure that the facts have been populated. For instance, make sure you have a play that talks to db_servers:

- hosts: db_servers
tasks:
- debug: msg="doesn't matter what you do, just that they were talked to previously."
Now you can use the facts within your template, like so:

{% for host in groups['db_servers'] %}
{{ hostvars[host]['ansible_eth0']['ipv4']['address'] }}
{% endfor %}

4.	What is the best way to see all of my host's inventory variables?

In order to check the inventory vars resulting from what you’ve defined in the inventory, you can execute the below command:

ansible -m debug -a "var=hostvars['hostname']" localhost

This will list down all the inventory vars.

5.	How should I set up a jump host to connect to servers that I don't have direct access to?

First step would be to set a ProxyCommand in the ansible_ssh_common_args inventory variable. All arguments that are defined in this variable are added to the sftp/scp/ssh command line when connecting to the relevant host. Let’s look at an example, consider the below inventory group:

[gatewayed]
foo ansible_host=192.0.2.1
bar ansible_host=192.0.2.2
Next, you can create group_vars/gatewayed.yml containing the following:

ansible_ssh_common_args: '-o ProxyCommand="ssh -W %h:%p -q user@gateway.example.com"'

Ansible will then append these arguments to the command line while trying to connect to any hosts in the group gatewayed.

6.	How do you deal with different machines that require different user accounts or ports to log in?

The simplest way to do this is by setting inventory variables in the inventory file.

For eg these hosts have different usernames and ports:

[webservers]
asdf.example.com ansible_port=5000 ansible_user=alice
jkl.example.com ansible_port=5001 ansible_user=bob
Also, if you wish to, you can specify the connection type to be used:

[testcluster]
localhost ansible_connection=local
/path/to/chroot1 ansible_connection=chroot
foo.example.com ansible_connection=paramiko
To make this more clear it is best to keep these in group variables or file them in a group_vars/<group-name> file.

7.	Assume you're configuring the production environment with Ansible, and your playbook uses an encrypted file. Passwords must be entered while using encrypted files. Can this process be automated, though, given Ansible is utilized for automation?

Yes, Ansible uses a feature called password file, where all the passwords to your encrypted files can be saved. So each time the user is asked for the password, he can simply make a call to the password file. The password is automatically read and entered by Ansible.

$ ansible-playbook launch.yml --vault-password-file ~/ .vault_pass.txt

Having a separate script that specifies the passwords is also possible. You need to make sure the script file is executable and the password is printed to standard output for it to work without annoying errors.

$ ansible-playbook launch.yml --vault-password-file ~/ .vault_pass.py



8. Is Ansible a free and open-source software?

Yes, Ansible is open source. That means you take the modules and rewrite them. Ansible is an open-source automated engine that lets you automate apps.